In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import scipy
import scipy.stats as stats

In [2]:
### 한글 폰트 설치 - 윈도우 사용할 때
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform

path = 'c:/Windows/Fonts/malgun.ttf'  # 또는 나눔고딕: 'c:/Windows/Fonts/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_sample = pd.read_csv("../datasets/sample_submission.csv")
df_train = pd.read_csv("../datasets/train.csv")
df_test = pd.read_csv("../datasets/test.csv")

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2452 non-null   object 
 1   매물확인방식  2452 non-null   object 
 2   보증금     2452 non-null   float64
 3   월세      2452 non-null   int64  
 4   전용면적    1665 non-null   float64
 5   해당층     2223 non-null   float64
 6   총층      2436 non-null   float64
 7   방향      2452 non-null   object 
 8   방수      2436 non-null   float64
 9   욕실수     2434 non-null   float64
 10  주차가능여부  2452 non-null   object 
 11  총주차대수   1756 non-null   float64
 12  관리비     2452 non-null   int64  
 13  중개사무소   2452 non-null   object 
 14  제공플랫폼   2452 non-null   object 
 15  게재일     2452 non-null   object 
 16  허위매물여부  2452 non-null   int64  
dtypes: float64(7), int64(3), object(7)
memory usage: 325.8+ KB


## 가설 검증 - 각 컬럼과 허위매물여부에 대한 다변수 검증 실시

- #### p_value < alpha : 귀무가설 기각
- #### p_value > alpha : 귀무가설 채택

### 1. 범주형(매물확인방식, 중개사무소, 제공플랫폼)

In [6]:
# 귀무가설 : 매물확인방식과 허위매물여부는 관련성이 없다.

from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_train['매물확인방식'], df_train['허위매물여부'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print("카이제곱 통계량:", chi2)
print("p-값:", p)
# print("자유도:", dof)
# print("기대 빈도:\n", expected)

# p-값 해석
alpha = 0.05
if p < alpha:
    print("매물확인방식과 허위매물여부 간의 관계가 있습니다.") 
else:
    print("매물확인방식과 허위매물여부 간의 관계가 없습니다.")

카이제곱 통계량: 5.250952693371378
p-값: 0.07240525881328504
매물확인방식과 허위매물여부 간의 관계가 없습니다.


In [7]:
# 귀무가설 : 중개사무소과 허위매물여부는 관련성이 없다.

from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_train['중개사무소'], df_train['허위매물여부'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print("카이제곱 통계량:", chi2)
print("p-값:", p)
# print("자유도:", dof)
# print("기대 빈도:\n", expected)

# p-값 해석
alpha = 0.05
if p < alpha:
    print("중개사무소와 허위매물여부 간의 관계가 있습니다.") 
else:
    print("중개사무소와 허위매물여부 간의 관계가 없습니다.")

카이제곱 통계량: 524.4480695707881
p-값: 2.3719684314814195e-17
중개사무소와 허위매물여부 간의 관계가 있습니다.


In [8]:
# 귀무가설 : 제공플랫폼과 허위매물여부는 관련성이 없다.

from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_train['제공플랫폼'], df_train['허위매물여부'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print("카이제곱 통계량:", chi2)
print("p-값:", p)
# print("자유도:", dof)
# print("기대 빈도:\n", expected)

# p-값 해석
alpha = 0.05
if p < alpha:
    print("제공플랫폼과 허위매물여부 간의 관계가 있습니다.") 
else:
    print("제공플랫폼과 허위매물여부 간의 관계가 없습니다.")

카이제곱 통계량: 19.920580943377207
p-값: 0.06860329781525687
제공플랫폼과 허위매물여부 간의 관계가 없습니다.


### 2. 연속형(보증금, 월세, 관리비) 
- #### 정규성 확인 - shapiro(5000개 미만), Anderson Test(5000개 이상)
    +  훈련데이터셋 2452개이므로 shapiro test 실시
- #### 정규분포 : Levene 검증
- #### 비정규분포 : 맨-휘트니 U(target 값 범주형 2집단), Krustal test(target 값 범주형 3집단)
    + 연속형 컬럼 모두 비정규 분포
    + 허위매물여부는 정상 or 허위 여부이므로 맨-휘트니 U 실시

In [9]:
from scipy.stats import shapiro

stat, p_value = shapiro(df_train["보증금"])

# 결과 출력
print("shapiro 검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("보증금 데이터는 정규분포를 따르지 않습니다.")
else:
    print("보증금 데이터는 정규분포를 따릅니다.")

shapiro 검정 통계량: 0.8774800642287577
p-값: 2.846647555613194e-40
보증금 데이터는 정규분포를 따르지 않습니다.


In [10]:
from scipy.stats import shapiro

stat, p_value = shapiro(df_train["월세"])

# 결과 출력
print("shapiro 검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("월세 데이터는 정규분포를 따르지 않습니다.")
else:
    print("월세 데이터는 정규분포를 따릅니다.")

shapiro 검정 통계량: 0.9612853865369135
p-값: 4.318400296706643e-25
월세 데이터는 정규분포를 따르지 않습니다.


In [11]:
from scipy.stats import shapiro

stat, p_value = shapiro(df_train["관리비"])

# 결과 출력
print("shapiro 검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("관리비 데이터는 정규분포를 따르지 않습니다.")
else:
    print("관리비 데이터는 정규분포를 따릅니다.")

shapiro 검정 통계량: 0.8280585968284627
p-값: 1.8744142408454236e-45
관리비 데이터는 정규분포를 따르지 않습니다.


In [15]:
from scipy.stats import mannwhitneyu

# 허위매물 여부에 따른 그룹 나누기
normal_group = df_train[df_train["허위매물여부"] == 0]["보증금"]
flaud_group = df_train[df_train["허위매물여부"] == 1]["보증금"]

# 맨-휘트니 U 검정 수행
stat, p_value = mannwhitneyu(normal_group, flaud_group)

# 결과 출력
print("검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("귀무가설을 기각합니다. 보증금은 허위매물여부에 유의미한 데이터입니다.")
else:
    print("귀무가설을 기각하지 않습니다. 보증금은 허위매물여부에 유의미하지 않는 데이터입니다.")

검정 통계량: 309766.0
p-값: 0.32908056378533024
귀무가설을 기각하지 않습니다. 보증금은 허위매물여부에 유의미하지 않는 데이터입니다.


In [16]:
from scipy.stats import mannwhitneyu

# 허위매물 여부에 따른 그룹 나누기
normal_group = df_train[df_train["허위매물여부"] == 0]["월세"]
flaud_group = df_train[df_train["허위매물여부"] == 1]["월세"]

# 맨-휘트니 U 검정 수행
stat, p_value = mannwhitneyu(normal_group, flaud_group)

# 결과 출력
print("검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("귀무가설을 기각합니다. 월세는 허위매물여부에 유의미한 데이터입니다.")
else:
    print("귀무가설을 기각하지 않습니다. 월세는 허위매물여부에 유의미하지 않는 데이터입니다.")

검정 통계량: 309182.0
p-값: 0.3043918342277385
귀무가설을 기각하지 않습니다. 월세는 허위매물여부에 유의미하지 않는 데이터입니다.


In [17]:
from scipy.stats import mannwhitneyu

# 허위매물 여부에 따른 그룹 나누기
normal_group = df_train[df_train["허위매물여부"] == 0]["관리비"]
flaud_group = df_train[df_train["허위매물여부"] == 1]["관리비"]

# 맨-휘트니 U 검정 수행
stat, p_value = mannwhitneyu(normal_group, flaud_group)

# 결과 출력
print("검정 통계량:", stat)
print("p-값:", p_value)

# p-값 해석
alpha = 0.05
if p_value < alpha:
    print("귀무가설을 기각합니다. 관리비는 허위매물여부에 유의미한 데이터입니다.")
else:
    print("귀무가설을 기각하지 않습니다. 관리비는 허위매물여부에 유의미하지 않는 데이터입니다.")

검정 통계량: 249408.5
p-값: 2.220315004089524e-10
귀무가설을 기각합니다. 관리비는 허위매물여부에 유의미한 데이터입니다.


- 유의미하다 ->  해당 연속형 컬럼에 대한 분포가 정상, 허위 둘다 다르다.
- 유의미하지않다 ->  해당 연속형 컬럼에 대한 분포가 정상, 허위 둘다 비슷함

### 인사이트 결과
1. 범주형과의 상관성 분석 결과
    - 중개사무소와 허위매물여부와의 상관성이 있음. -> 유의미한 데이터
    - 중개사무소에 따른 허위매물 판단을 할 수 있음.
    - 플랫폼이나 매물방식에 대해서는 관련 x
2. 연속형과의 상관성 분석 결과
    - 연속형 컬럼들은 비정규분포를 따름.
    - Mann Whitney U 방식 검증을 통해 상관성 도출
    - 관리비는 정상 매물과 허위 매물 간 분포값을 다르게 보임. -> 유의미한 데이터
    - 허위매물을 발생시키는 지표로 판단됨.